# Setup for Amazon Bedrock
* Container: `conda_python3` <BR>
* We recommend `python 3.10` or later. 
    - version check: !python -V

## 0. Materials
- Bedrock user guide
    - https://docs.aws.amazon.com/bedrock/latest/userguide/what-is-service.html
- Step by step vidio tutorial
    - https://www.youtube.com/watch?v=ab1mbj0acDo

## 1. role setting (adding trust relationship)

### 1.1. role check

In [ ]:
from sagemaker import get_execution_role

In [ ]:
strSageMakerRoleName = get_execution_role().rsplit('/', 1)[-1]
print (f"SageMaker Execution Role Name: {strSageMakerRoleName}")

### 1.2. policy
- 1.1에서 확인된 롤에 아래와 같이 3개의 권한 추가
    - AmazonBedrockFullAccess
    - AmazonOpenSearchServiceFullAccess
    - AmazonSSMFullAccess

## 2. Model access

### 2.1. Amazon Bedrock Console
![nn](../imgs/model-access/1.png)

### 2.2. "Get Started"
![nn](../imgs/model-access/2.png)

### 2.3. "Model access" - "Edit"
사용 모델 활성화 창
![nn](../imgs/model-access/3.png)

### 2.3. "Save Changes"
사용할 모델을 활성화 후 저장
![nn](../imgs/model-access/4.png)

## 3. Install python SDK for bedrock

In [ ]:
install_needed = True

In [ ]:
import sys
import IPython

if install_needed:
    print("installing deps and restarting kernel")
    !{sys.executable} -m pip install -U pip
    !{sys.executable} -m pip install -U awscli
    !{sys.executable} -m pip install -U botocore
    !{sys.executable} -m pip install -U boto3
    !{sys.executable} -m pip install -U sagemaker 
    !{sys.executable} -m pip install -U langchain
    !{sys.executable} -m pip install -U langchain-community
    !{sys.executable} -m pip install -U termcolor
    !{sys.executable} -m pip install -U transformers
    !{sys.executable} -m pip install -U librosa
    !{sys.executable} -m pip install -U opensearch-py
    !{sys.executable} -m pip install -U sqlalchemy #==2.0.1
    !{sys.executable} -m pip install -U pypdf
    !{sys.executable} -m pip install -U spacy
    !{sys.executable} -m spacy download ko_core_news_md
    !{sys.executable} -m pip install -U ipython
    !{sys.executable} -m pip install -U ipywidgets
    !{sys.executable} -m pip install -U llmsherpa
    !{sys.executable} -m pip install -U anthropic
    !{sys.executable} -m pip install -U faiss-cpu
    !{sys.executable} -m pip install -U jq

    IPython.Application.instance().kernel.do_shutdown(True)

## 4. Check setting
모델 리스트 <BR>
![nn](../imgs/check.png)


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
module_path = ".."
sys.path.append(os.path.abspath(module_path))

In [ ]:
import boto3
import awscli
import botocore
import langchain
from pprint import pprint
from termcolor import colored
from utils.bedrock import bedrock_info

In [ ]:
print(f"langchain version check: {langchain.__version__}")
print(f"boto3 version check: {boto3.__version__}")
print(f"botocore version check: {botocore.__version__}")
print(f"awscli version check: {awscli.__version__}")

**Select region: "us-east-1"(M1), "us-west-2"(M2)**

In [ ]:
#bedrock_region = "us-east-1"
bedrock_region = "us-west-2"

In [ ]:
bedrock_client = boto3.client(
    service_name='bedrock-runtime',
    region_name=bedrock_region,
    endpoint_url=f"https://bedrock-runtime.{bedrock_region}.amazonaws.com"
)

In [ ]:
print (colored("\n== FM lists ==", "green"))
pprint (bedrock_info.get_list_fm_models(verbose=False))

## 5. generation

In [ ]:
from langchain.llms.bedrock import Bedrock
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

### Titan

## Titan text모델은 현재 preview 서비스입니다. GA 이후 업데이트 하도록 할께요! (현재 에러발생)

In [ ]:
llm = Bedrock(
    model_id=bedrock_info.get_model_id(model_name="Titan-Text-G1"),
    client=bedrock_client,
    model_kwargs={
        "maxTokenCount":512,
        "stopSequences":[],
        "temperature":0,
        "topP":0.9
    },
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

In [ ]:
prompt = "Please let us know SageMaker's advantages in 100 words"

In [ ]:
print (colored(llm(prompt), "green"))

### Claude v2.1

In [ ]:
llm = Bedrock(
    model_id=bedrock_info.get_model_id(model_name="Claude-V2-1"),
    client=bedrock_client,
    model_kwargs={
        "max_tokens_to_sample":512,
        "stop_sequences":["\n\nHuman:"],
        "temperature":0,
        "top_p":0.9
    },
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

In [ ]:
prompt = "\n\nHuman: 세이지메이커의 장점을 100단어로 알려주세요 \n\nAssistant:"

In [ ]:
response = llm(prompt)

* Long text to examine 200k-input (optional)

In [ ]:
text_file = "../utils/sample-text/long-text-70k-tokens.txt"
with open(text_file, "r") as file:
    text = file.read()
    
prompt = "".join(["\n\nHuman: 아래 글을 요약해줘\n\n", text, "\n\nAssistant:"])
    
print (f"# characters in prompt: {len(prompt)}")

In [ ]:
# response = llm(prompt)